# Multi-Horizon Financial Forecasting on IPU using DeepLOB-Attention - Training

The [original Jupyter notebook](https://github.com/zcakhaa/Multi-Horizon-Forecasting-for-Limit-Order-Books/blob/main/code_gpu/run_train_deeplob_attention.ipynb) was based on the paper: ["Multi-Horizon Forecasting for Limit Order Books: Novel Deep Learning Approaches and Hardware Acceleration using Intelligent Processing Units"](https://arxiv.org/abs/2105.10430).
Original authors: Zihao Zhang and Stefan Zohren
Institute: Oxford-Man Institute of Quantitative Finance, Department of Engineering Science, University of Oxford
Copyright (c) 2021 Oxford Man Institute & University of Oxford. All rights reserved.


Copyright (c) 2023 Graphcore Ltd. All rights reserved.
This Jupyter notebook has been modified by Graphcore Ltd so that it can be run with the latest version of Graphcore's [Poplar (TM) SDK](https://docs.graphcore.ai/projects/sdk-overview/).

On Paperspace:

|  Domain | Tasks | Model | Datasets | Workflow |   Number of IPUs   | Execution time |
|---------|-------|-------|----------|----------|--------------|--------------|
| Finance | prediction  | DeepLOB-Attention |  Limit-Order Books (FI-2010) | training, inference | 4 | ~6 minutes  |

**This notebook demonstrates how to train the DeepLOB-Attention model on Graphcore IPUs using TensorFlow 2.**

[![Join our Slack Community](https://img.shields.io/badge/Slack-Join%20Graphcore's%20Community-blue?style=flat-square&logo=slack)](https://www.graphcore.ai/join-community)

## Methods

This Jupyter notebook is used to demonstrate the machine learning methods for multi-horizon forecasting for limit order books, shown in [2], implemented using TensorFlow 2. The publicly available FI-2010 [1] dataset is used for model training, validation and inference.

## Data
The FI-2010 dataset is publicly available and interested readers can check out the paper [1]. The dataset can be downloaded from: https://etsin.fairdata.fi/dataset/73eb48d7-4dbc-4a10-a52a-da745b47a649 

This notebook will download the data automatically. Alternatively, it may be obtained at the following URL: 

https://drive.google.com/drive/folders/1Xen3aRid9ZZhFqJRgEMyETNazk02cNmv?usp=sharing.

## References
[1] Ntakaris A, Magris M, Kanniainen J, Gabbouj M, Iosifidis A. Benchmark dataset for mid‐price forecasting of limit order book data with machine learning methods. Journal of Forecasting. 2018 Dec;37(8):852-66. https://arxiv.org/abs/1705.03233

[2] Zhang Z, Zohren S. Multi-Horizon Forecasting for Limit Order Books: Novel Deep Learning Approaches and Hardware Acceleration using Intelligent Processing Units. https://arxiv.org/abs/2105.10430



## Environment setup

The best way to run this demo is on Paperspace Gradient’s cloud IPUs because everything is already set up for you. To improve your experience, we preload datasets and pre-install packages. This can take a few minutes. If you experience errors immediately after starting a session, please try restarting the kernel before contacting support. If a problem persists or you want to give us feedback on the content of this notebook, please reach out to through our community of developers using our [Slack channel](https://www.graphcore.ai/join-community) or raise a [GitHub issue](https://github.com/graphcore/Gradient-Tensorflow2/issues).

[![Run on Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://ipu.dev/MUD3GR)

To run the demo using other IPU hardware, you need to have the Poplar SDK enabled. Refer to the [Getting Started guide](https://docs.graphcore.ai/en/latest/getting-started.html#getting-started) for your system for details on how to enable the Poplar SDK. Also refer to the [Jupyter Quick Start guide](https://docs.graphcore.ai/projects/jupyter-notebook-quick-start/en/latest/index.html) for how to set up Jupyter to be able to run this notebook on a remote IPU machine.

## Install dependencies

First we need to install the required Python libraries.

In [ ]:
%pip install -r requirements.txt

## Import the necessary libraries

Next we import the Python libraries necessary for running training, validation and inference.

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import logging
import glob
import argparse
import sys
import time
import tensorflow as tf
from tensorflow.python import ipu
from tensorflow import keras
from tensorflow.keras import backend as K
from ipu_tensorflow_addons.keras.layers import LSTM
from sklearn.metrics import accuracy_score, classification_report
import pickle
import numpy as np
from collections import Counter
import zipfile

## Download the dataset

Now we download the dataset, which should take only a few moments.

In [ ]:
if not os.path.isfile("data.zip"):
    !wget https://raw.githubusercontent.com/zcakhaa/DeepLOB-Deep-Convolutional-Neural-Networks-for-Limit-Order-Books/master/data/data.zip
    print("Data downloaded.")
else:
    print("Data already exists, skipping download.")

with zipfile.ZipFile("data.zip", "r") as zip_ref:
    zip_ref.extractall("./")

## Configuration

We set a number of variables relating to model size and hyperparameters, as well as the location for saving the trained model checkpoints. We also set the number of epochs for which we wish to train the model.

For full training, it is recommended to set the number of epochs to >= 150 as per the original paper.

In [ ]:
T = 50  # lookback window size
epochs = 20  # number of training epochs
batch_size = 16  # gradient descent batch size
n_hidden = 64  # hidden state for decoder
SHUFFLE = True  # shuffle the traning data
saved_model_path = os.getenv("CHECKPOINT_DIR", "/tmp/checkpoints")
saved_model_path = os.path.join(saved_model_path, "deeplob_attention")

Let's also set an environment variable which allows to use the executable caches, saving us from recompiling the model.

In [ ]:
executable_cache_dir = os.getenv("POPLAR_EXECUTABLE_CACHE_DIR", "/tmp/exe_cache/")
os.environ["TF_POPLAR_FLAGS"] = f"--executable_cache_path='{executable_cache_dir}'"

## Data processing

Next we define a number of functions which process the data on the host before it is sent to the IPU.

In [ ]:
def prepare_x(data):
    df1 = data[:40, :].T
    return np.array(df1)

In [ ]:
def get_label(data):
    lob = data[-5:, :].T
    all_label = []

    for i in range(lob.shape[1]):
        one_label = lob[:, i] - 1
        one_label = keras.utils.to_categorical(one_label, 3)
        one_label = one_label.reshape(len(one_label), 1, 3)
        all_label.append(one_label)

    return np.hstack(all_label)

In [ ]:
def data_classification(X, Y, T):
    [N, D] = X.shape
    df = np.array(X)

    dY = np.array(Y)

    dataY = dY[T - 1 : N]

    dataX = np.zeros((N - T + 1, T, D))
    for i in range(T, N + 1):
        dataX[i - T] = df[i - T : i, :]

    return dataX.reshape(dataX.shape + (1,)), dataY

In [ ]:
def prepare_decoder_input(data, teacher_forcing):
    if teacher_forcing:
        first_decoder_input = keras.utils.to_categorical(np.zeros(len(data)), 3)
        first_decoder_input = first_decoder_input.reshape(
            len(first_decoder_input), 1, 3
        )
        decoder_input_data = np.hstack((data[:, :-1, :], first_decoder_input))

    if not teacher_forcing:
        decoder_input_data = np.zeros((len(data), 1, 3))
        decoder_input_data[:, 0, 0] = 1.0

    return decoder_input_data

## Load data from disk

Now we load the dataset from the disk into NumPy arrays and pre-process the data using the functions we defined.

In [ ]:
# load data
dec_train = np.loadtxt("Train_Dst_NoAuction_DecPre_CF_7.txt")
dec_test1 = np.loadtxt("Test_Dst_NoAuction_DecPre_CF_7.txt")
dec_test2 = np.loadtxt("Test_Dst_NoAuction_DecPre_CF_8.txt")
dec_test3 = np.loadtxt("Test_Dst_NoAuction_DecPre_CF_9.txt")
dec_test = np.hstack((dec_test1, dec_test2, dec_test3))

# extract limit order book data from the FI-2010 dataset
train_lob = prepare_x(dec_train)
test_lob = prepare_x(dec_test)

# extract label from the FI-2010 dataset
train_label = get_label(dec_train)
test_label = get_label(dec_test)

# prepare training data. We feed past T observations into our algorithms.
train_encoder_input, train_decoder_target = data_classification(
    train_lob, train_label, T
)
train_decoder_input = prepare_decoder_input(train_encoder_input, teacher_forcing=False)

test_encoder_input, test_decoder_target = data_classification(test_lob, test_label, T)
test_decoder_input = prepare_decoder_input(test_encoder_input, teacher_forcing=False)

print(
    f"train_encoder_input.shape = {train_encoder_input.shape},"
    f"train_decoder_target.shape = {train_decoder_target.shape}"
)
print(
    f"test_encoder_input.shape = {test_encoder_input.shape},"
    f"test_decoder_target.shape = {test_decoder_target.shape}"
)

## IPU configuration

In order to use the IPUs, we create an IPU configuration using the `IPUConfig` class.

For this model only one IPU is required to perform training, validation and inference. However by requesting multiple IPUs we can increase throughput by executing tasks in a data parallel fashion.

The model is replicated across the IPUs, with an identical copy of the model residing on each IPU. Each IPU receives a different batch of input data, and during training the gradients are automatically averaged across the replicas before the backward pass is performed.

By setting `ipu_config.auto_select_ipus = num_ipus` we can automatically select the first available device containing the desired number of IPUs on the system:

In [ ]:
# Number of IPUs over which to replicate the model
available_ipus = int(os.getenv("NUM_AVAILABLE_IPU", 4))
num_ipus = min(available_ipus, 4)  # Not intended to scale beyond POD4

# Configure the IPU system
ipu_config = ipu.config.IPUConfig()
ipu_config.auto_select_ipus = num_ipus
ipu_config.configure_ipu_system()

## Create the model

Now we create the DeepLOB-Attention model, from [Luong et al](https://arxiv.org/abs/1508.04025), using the Keras functional API.

The attention model is an evolution of the seq2seq model shown in the DeepLOBSeq2Seq notebook, designed to better handle inputs containing long sequences.

Multiple `Input` layers are present since the encoder and decoder parts of the model take separate input tensors, and so when constructing the dataset, this will need to be considered.

In [ ]:
def get_model_attention(latent_dim):
    # Luong Attention
    # https://arxiv.org/abs/1508.04025

    input_train = keras.Input(shape=(50, 40, 1))

    conv_first1 = keras.layers.Conv2D(32, (1, 2), strides=(1, 2))(input_train)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
    conv_first1 = keras.layers.Conv2D(32, (4, 1), padding="same")(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
    conv_first1 = keras.layers.Conv2D(32, (4, 1), padding="same")(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)

    conv_first1 = keras.layers.Conv2D(32, (1, 2), strides=(1, 2))(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
    conv_first1 = keras.layers.Conv2D(32, (4, 1), padding="same")(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
    conv_first1 = keras.layers.Conv2D(32, (4, 1), padding="same")(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)

    conv_first1 = keras.layers.Conv2D(32, (1, 10))(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
    conv_first1 = keras.layers.Conv2D(32, (4, 1), padding="same")(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)
    conv_first1 = keras.layers.Conv2D(32, (4, 1), padding="same")(conv_first1)
    conv_first1 = keras.layers.LeakyReLU(alpha=0.01)(conv_first1)

    # build the inception module
    convsecond_1 = keras.layers.Conv2D(64, (1, 1), padding="same")(conv_first1)
    convsecond_1 = keras.layers.LeakyReLU(alpha=0.01)(convsecond_1)
    convsecond_1 = keras.layers.Conv2D(64, (3, 1), padding="same")(convsecond_1)
    convsecond_1 = keras.layers.LeakyReLU(alpha=0.01)(convsecond_1)

    convsecond_2 = keras.layers.Conv2D(64, (1, 1), padding="same")(conv_first1)
    convsecond_2 = keras.layers.LeakyReLU(alpha=0.01)(convsecond_2)
    convsecond_2 = keras.layers.Conv2D(64, (5, 1), padding="same")(convsecond_2)
    convsecond_2 = keras.layers.LeakyReLU(alpha=0.01)(convsecond_2)

    convsecond_3 = keras.layers.MaxPooling2D((3, 1), strides=(1, 1), padding="same")(
        conv_first1
    )
    convsecond_3 = keras.layers.Conv2D(64, (1, 1), padding="same")(convsecond_3)
    convsecond_3 = keras.layers.LeakyReLU(alpha=0.01)(convsecond_3)

    convsecond_output = keras.layers.concatenate(
        [convsecond_1, convsecond_2, convsecond_3], axis=3
    )
    conv_reshape = keras.layers.Reshape(
        (int(convsecond_output.shape[1]), int(convsecond_output.shape[3]))
    )(convsecond_output)

    # seq2seq
    encoder_inputs = conv_reshape
    encoder = LSTM(latent_dim, return_state=True, return_sequences=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    states = [state_h, state_c]

    # Set up the decoder, which will only process one timestep at a time.
    decoder_inputs = keras.Input(shape=(1, 3))
    decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
    decoder_dense = keras.layers.Dense(3, activation="softmax", name="output_layer")

    all_outputs = []
    all_attention = []

    encoder_state_h = keras.layers.Reshape((1, int(state_h.shape[1])))(state_h)
    inputs = keras.layers.concatenate([decoder_inputs, encoder_state_h], axis=2)

    for _ in range(5):
        # h'_t = f(h'_{t-1}, y_{t-1}, c)
        outputs, state_h, state_c = decoder_lstm(inputs, initial_state=states)
        # dot
        attention = keras.layers.dot([outputs, encoder_outputs], axes=2)
        attention = keras.layers.Activation("softmax")(attention)
        # context vector
        context = keras.layers.dot([attention, encoder_outputs], axes=[2, 1])
        context = keras.layers.BatchNormalization(momentum=0.6)(context)

        # y = g(h'_t, c_t)
        decoder_combined_context = keras.layers.concatenate([context, outputs])
        outputs = decoder_dense(decoder_combined_context)
        all_outputs.append(outputs)
        all_attention.append(attention)

        inputs = keras.layers.concatenate([outputs, context], axis=2)
        states = [state_h, state_c]

    # Concatenate all predictions
    decoder_outputs = keras.layers.Lambda(
        lambda x: K.concatenate(x, axis=1), name="outputs"
    )(all_outputs)
    decoder_attention = keras.layers.Lambda(
        lambda x: K.concatenate(x, axis=1), name="attentions"
    )(all_attention)

    # Define and compile model as previously
    model = keras.Model(inputs=[input_train, decoder_inputs], outputs=decoder_outputs)
    return model

## Dataset creation

Having constructed the Keras model, the next step is to define the construction of the datasets for training, validation and inference.

We define a dataset creation function that produces a TensorFlow `Dataset` object from the input tensors, casts to the `float32` datatype and then batches and optionally shuffles/repeats the dataset depending on whether it is intended for model training, validation or inference.

In [ ]:
def create_dataset(
    encoder_input, decoder_input, encoder_target, batch_size, method, shuffle=False
):
    train_pairs_ds = tf.data.Dataset.from_tensor_slices((encoder_input, decoder_input))
    train_pairs_ds = train_pairs_ds.map(
        lambda d, l: (tf.cast(d, tf.float32), tf.cast(l, tf.float32))
    )

    train_y_ds = tf.data.Dataset.from_tensor_slices(encoder_target)
    train_y_ds = train_y_ds.map(lambda d: (tf.cast(d, tf.float32)))

    if method != "prediction":
        train_ds = tf.data.Dataset.zip((train_pairs_ds, train_y_ds))

        if shuffle:
            train_ds = train_ds.shuffle(len(encoder_input))
        train_ds = train_ds.batch(batch_size, drop_remainder=True)

    if method == "train":
        return train_ds.repeat()

    if method == "val":
        return train_ds

    if method == "prediction":
        test_ds = tf.data.Dataset.from_tensor_slices((encoder_input, decoder_input))
        test_ds = test_ds.batch(batch_size, drop_remainder=True)
        test_ds = test_ds.map(
            lambda d, l: [(tf.cast(d, tf.float32), tf.cast(l, tf.float32))]
        )

        return test_ds

## Steps per execution

The IPU can perform multiple training, validation or inference steps in an on-device loop for a single call to the model. Since this reduces the amount of host-IPU communication, this can allow for a much better throughput for a given process.

The number of steps executed in such a loop per call to the underlying hardware is controlled by the `steps_per_execution` variable.

Setting this value greater than 1 typically improves performance, whilst having no effect on how the model trains in terms of weight updates. However, we need to ensure the number of batches in the dataset is divisible by this number.

We therefore define a helper function to calculate the maximum permissible values for `steps_per_epoch`, `validation_steps` and `test_steps` such that the dataset has enough data for each epoch of training and to run model validation and inference.

In [ ]:
steps_per_execution = 100

In [ ]:
def make_dataset_divisible(num_elements, batch_size, steps_per_exe, num_replicas):
    return (
        num_elements
        // batch_size
        // steps_per_exe
        // num_replicas
        * num_replicas
        * steps_per_exe
    )

## Train the model

We can now train our model using the Keras `model.fit()` API.

An `IPUStrategy` class is created, with model construction and execution done from within the scope of this strategy in order to target the IPU.

The `steps_per_execution` argument is added to `model.compile()`, along with the loss, optimiser and metrics.

We train the model nominally in chunks of 5 epochs. At the end of each set of 5 epochs, we perform model validation to ensure the model is not overfitting to the training dataset.

Once the model has trained for the desired number of epochs, the model checkpoint is saved to the directory we specified earlier.

In [ ]:
strategy = ipu.ipu_strategy.IPUStrategy()
all_results = [[1000, 0]]
split_train_val = int(np.floor(len(train_encoder_input) * 0.8))

with strategy.scope():
    # Create an instance of the model
    model = get_model_attention(n_hidden)

    # Get the dataset
    train_ds = create_dataset(
        train_encoder_input[:split_train_val],
        train_decoder_input[:split_train_val],
        train_decoder_target[:split_train_val],
        batch_size,
        method="train",
        shuffle=SHUFFLE,
    )
    val_ds = create_dataset(
        train_encoder_input[split_train_val:],
        train_decoder_input[split_train_val:],
        train_decoder_target[split_train_val:],
        batch_size,
        method="val",
    )
    test_ds = create_dataset(
        test_encoder_input,
        test_decoder_input,
        test_decoder_target,
        batch_size,
        method="prediction",
    )

    # Train the model
    adam = keras.optimizers.Adam(learning_rate=0.00004, beta_1=0.9, beta_2=0.999)

    model.compile(
        loss="categorical_crossentropy",
        metrics=["accuracy"],
        optimizer=adam,
        steps_per_execution=steps_per_execution,
    )

    epoch_ = 0
    epochs_per_fit = 5

    train_batches = len(train_encoder_input[:split_train_val])
    val_batches = len(train_encoder_input[split_train_val:])
    test_batches = len(test_encoder_input)

    steps_per_epoch = make_dataset_divisible(
        train_batches, batch_size, steps_per_execution, num_ipus
    )
    val_steps = make_dataset_divisible(
        val_batches, batch_size, steps_per_execution, num_ipus
    )
    test_steps = make_dataset_divisible(
        test_batches, batch_size, steps_per_execution, num_ipus
    )

    while epoch_ < epochs:

        model.fit(
            train_ds,
            steps_per_epoch=steps_per_epoch,
            initial_epoch=epoch_,
            epochs=epoch_ + epochs_per_fit,
        )

        epoch_ = epoch_ + epochs_per_fit
        result = model.evaluate(
            val_ds,
            steps=val_steps,
        )

        all_results.append(result)
        print(f"Epoch = {epoch_}," f"Validation Results = {result}")

        if all_results[-1][0] < all_results[-2][0]:
            model.save_weights(saved_model_path)

## Model inference

Having trained the model, or simply reloaded the model weights from a checkpoint generated previously, we can now run inference over the test dataset.

Again, we ensure the model is called from within `strategy.scope()` in order to target the IPU.

In [ ]:
with strategy.scope():
    model.load_weights(saved_model_path)
    pred = model.predict(test_ds, steps=test_steps)

## Results

Finally we can compare the output obtained from inference with the ground truth values, to see how the model performs. We define a helper function which calculates and prints the accuracy score using the SciKit-Learn library, as well as other metrics of interest, for each prediction horizon.

In [ ]:
def evaluation_metrics(real_y, pred_y):
    real_y = real_y[: len(pred_y)]
    logging.info("-------------------------------")

    for i in range(real_y.shape[1]):
        print(f"Prediction horizon = {i}")
        print(
            f"accuracy_score = {accuracy_score(np.argmax(real_y[:, i], axis=1), np.argmax(pred_y[:, i], axis=1))}"
        )
        print(
            f"classification_report = {classification_report(np.argmax(real_y[:, i], axis=1), np.argmax(pred_y[:, i], axis=1), digits=4)}"
        )
        print("-------------------------------")

In [ ]:
evaluation_metrics(test_decoder_target, pred)
ipu.config.reset_ipu_configuration()

## Conclusion

We have demonstrated how you can use IPUs to run training and inference on the DeepLOB-Attention model with an accuracy of 80%.

Interested in other applications for TensorFlow 2 on the IPU? Check out our GNN notebooks:

* GPS++ model found in `/ogb-competition`
* Cluster GCN model found in `/gnn-cluster-gcn`

We also have tutorials dedicated to using IPUs with TensorFlow 2 which is located in `/learning-Tensorflow2-on-IPU` which includes an MNIST tutorial.